In [1]:
import torch
import numpy as np
import pandas as pd
import codecs
import re
import nltk
import random

from nltk.stem import WordNetLemmatizer

from random import shuffle

from collections import Counter

from numpy import array

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

from torch import tensor
from torch import nn
from torch import optim
from torch.autograd import Variable
import torch.utils.data.dataloader as dataloader
from torch.utils.data import Dataset


# Parameters

In [2]:
# Number of recognized words you put in input
nb_input = 100 #8414 max

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 5
hidden_size = 10

# Learning rate
lr = 0.001

# Number of epochs
nb_epochs = 1

# Random seed, don't change it if you don't know what it is
random_seed = 42

use_cuda = torch.cuda.is_available()

In [3]:
def seeding_random():
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    if use_cuda:
        torch.cuda.manual_seed_all(random_seed)
        torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.deterministic=True

In [4]:
# Lemmatisation
# Working on list_of_words
# TODO: it doesn't work
def lemm(phrase):
    lemmer = WordNetLemmatizer()
    lemmed = []
    for word in phrase:
        lemmed.append(lemmer.lemmatize(word, pos="v"))
    return lemmed

## Dataloader implementation

In [5]:
class QuestionDataset(Dataset):
    def __init__(self, file, nb_most_commons):
        # Encoding in windows-1252, utf-8 generate error on some char
        file = codecs.open(file, "r+","windows-1252")
        train_data = []
        for line in file.readlines():
            train_data.append(line)

        # Divided file into 2 list:
        # questions = list of questions 
        # labels = list of labels

        questions = []
        labels = []

        # Black list
        regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')

        (train_data[0]).split()[0]
        for string in train_data:
            question_str = []
            for x in lemm(string.split()[1:]):
                question_str.append(x.lower())
            labels.append(string.split()[0])
            questions.append(question_str)
        #print("len(questions) = " + str(len(questions)))
        #print("Exemples of questions:")
        #print(questions[:2])

        self.add_tag(questions)
        
        # Vocabulary of unique words
        data = []
        data.append('<unk>')
        for q in questions:
            for word in q:
                data.append(word)
        #print(len(data))
        scv = np.array(data)
        unik, counts = np.unique(scv,return_counts=True)
        vocabulary = {}
        for i in range(0, len(unik)):
            vocabulary[unik[i]] = counts[i]
    
        word_list = list([x[0] for x in Counter(vocabulary).most_common(nb_most_commons+2)])
        # We add the unk word for future purpose.
        word_list.append('<unk>')
        words_array = np.array(word_list)
        
        # Integer encoding with OneHotEncoder
        words_tre = words_array.reshape(len(words_array),1)
        one_hot_encoder = OneHotEncoder(sparse=False)
        onehot_encoded = one_hot_encoder.fit_transform(words_tre)
        # Creating a dictionnary of word and its one hot array
        self.words_onehoted = {}
        for i in range(0, len(words_array)):
            self.words_onehoted[word_list[i]] = onehot_encoded[i]

        # One hot categories
        self.categories_onehoted = {}
        self.categories_onehoted['ABBR'] = [1, 0, 0, 0, 0, 0] # Abbreviation
        self.categories_onehoted['ENTY'] = [0, 1, 0, 0, 0, 0] # Entity
        self.categories_onehoted['DESC'] = [0, 0, 1, 0, 0, 0] # Description
        self.categories_onehoted['HUM']  = [0, 0, 0, 1, 0, 0] # Human
        self.categories_onehoted['LOC']  = [0, 0, 0, 0, 1, 0] # Location
        self.categories_onehoted['NUM']  = [0, 0, 0, 0, 0, 1] # Numeric
        
        max_length = max([len(q) for q in questions])
        print(max_length)
        self.batch_data = []
        for num_question in range(len(questions)):
            # Construction of question_onehot list.
            zeros = [[0. for _ in range(nb_most_commons+3)] for x in range(max_length-len(questions[num_question]))]
            question_onehot = zeros+[self.get_onehot_word(word) for word in questions[num_question]]
            #print((question_onehot))
#Hey reg la len de question onehot
            # Construction of category_onehot.
            category = labels[num_question].partition(':')[0]
            category_onehot = self.get_onehot_category(category)
            #print(question_onehot)
            self.batch_data.append([tensor(question_onehot), tensor(category_onehot)])
    
        #self.batch_data = self.batch_data[:5000]
        
    
    # Function to get the corresponding one hot list for a category.
    def get_onehot_category(self, category):
        return self.categories_onehoted[category]


    # Function to get the corresponding one hot list for a word.
    def get_onehot_word(self, word):
        if word in self.words_onehoted:
            return list(self.words_onehoted[word])
        else:
            return list(self.words_onehoted['<unk>'])

    

    # Method to add tags begin and end to phrases list.
    # /!\ 
    # WARNING : this method need to be executed only ONE time.
    # /!\
    def add_tag(self, question_list):
        for i in range(0, len(question_list)):
            if question_list[i][0] != '<bos>' :
                question_list[i].insert(0, '<bos>')
                question_list[i].append('<eos>')
                
    def __len__(self):
        return len(self.batch_data)

    def __getitem__(self, idx):
        seeding_random()
        return self.batch_data[idx]

def pad_collate(batch):
    return batch
    #print(batch)
#    res = []
#    for num_question in range(len(batch)):
#        res.append([tensor(batch[num_question][0]), tensor(batch[num_question][1])])
#    return res
    

In [6]:

seeding_random()
training_set = QuestionDataset("train_all.label", nb_input-3)
seeding_random()

test_set = QuestionDataset("TREC_test.label", nb_input-3)
seeding_random()

print(training_set[0][0])
# Création du DataLoader
dataloader_args = dict(shuffle=True, batch_size=2, num_workers=1,
                       pin_memory=True, worker_init_fn=seeding_random()) if use_cuda else dict()
seeding_random()

train_loader = dataloader.DataLoader(training_set, **dataloader_args)
seeding_random()

test_loader = dataloader.DataLoader(test_set, **dataloader_args)
seeding_random()



39
19
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


# RNN implementation
Using ReLU, and CrossEntropy

In [7]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        
        # Applying RNN layer, and softmax then
        self.rnn = nn.RNN(input_size=nb_inputs, num_layers=nb_layers,
                   hidden_size=nb_neurons, dropout=0., batch_first=True, nonlinearity='relu')
        self.inter = nn.Linear(nb_neurons, nb_outputs)
        self.sm = nn.Softmax(dim=2)
        
        # Other usefull variables here
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.input_dim = nb_inputs
        self.output_dim = nb_output
        self.nb_layers = nb_layers
        self.nb_neurons = nb_neurons
        
    def forward(self, inputs):
        h0 = torch.zeros(self.nb_layers, inputs.size(0), self.nb_neurons)
        if use_cuda:
            h0 = h0.to("cuda")
        x, hn = self.rnn(inputs, h0)
        
        x = self.inter(x)
        x = self.sm(x)
        return x

# End of the class RNN

# Now let's define learn(), which learn a RNN some data
def learn(rnn, data_loader, num_epochs=1):
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        rnn.cuda(device)
    
    # Preparing
    rnn.train()
    losses = []
    criterion = nn.CrossEntropyLoss()
    

    for epoch in range(num_epochs):
        
        for batch_idx, (data, target) in enumerate(data_loader):
            #print(data)
            data, target = data.to(device), target.to(device)
            
            output = rnn(data)
            loss = criterion(output, target)
            losses.append(loss.data.item())
            rnn.optimizer.zero_grad()
            loss.backward()
            rnn.optimizer.step()
            
            # Print the progress
            if batch_idx % 100 == 0 or batch_idx % 100 == 1 or batch_idx == len(data_loader)-1:
                  print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                    epoch, 
                    batch_idx * len(data), 
                    len(data_loader.dataset),
                    100. * batch_idx / len(data_loader), 
                    loss.data.item()), 
                    end='')
        print()
        
    # Return losses list, you can print them later if you want
    return losses

# return (rightAnswer, falseAnswer)
def getEfficience(rnn, data_loader, tresh=0) :
    rightAnswer = 0
    falseAnswer = 0
    
    device = torch.device("cuda" if use_cuda else "cpu")
    for (data, target) in data_loader :
        data, target = data.to(device), target.to(device)
        predicted = rnn(data).detach().cpu().numpy()[-1][-1]
        
        if np.argmax(predicted) == np.argmax(target.detach().cpu().numpy()[-1]):
            rightAnswer += 1
        else:
            falseAnswer += 1
    return (rightAnswer, falseAnswer)

# Using the RNN

In [8]:
seeding_random()

rnn = RNN(nb_inputs = nb_input, nb_layers=nb_hidd_lay,
          nb_neurons=hidden_size, nb_outputs=nb_output, learning_rate=lr)
if use_cuda:
    rnn = rnn.to("cuda")

seeding_random()

losses = learn(rnn, train_loader, nb_epochs)
print("Done :)")

 Train Epoch: 0 [15450/15452 (100%)]	 Loss: 3.582700
Done :)


## Error curve

In [9]:
from scipy.signal import savgol_filter
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow


def update_losses(smooth=1):
    x = np.linspace(0, len(losses), len(losses))
    fig = plt.figure(figsize=(13, 8)) 
    ax = fig.add_subplot(1,1,1)
    cnn_line, = ax.plot(x, losses)
    cnn_line.set_ydata(savgol_filter(losses, smooth, 3))
    
interact(update_losses, smooth=(5, 201, 2));


interactive(children=(IntSlider(value=5, description='smooth', max=201, min=5, step=2), Output()), _dom_classe…

# Analysis on test set

In [10]:


rnn.eval()

seeding_random()


final_results = getEfficience(rnn, train_loader)
total = sum(final_results)

correct = final_results[0]/total*100
false = final_results[1]/total*100



print("Congratulations!")
print("On the training set:")
print("Corrects: " + str(correct) + "%")
print("False:    " + str(false) + "%")
print()

seeding_random()

final_results = getEfficience(rnn, test_loader)
total = sum(final_results)

correct = final_results[0]/total*100
false = final_results[1]/total*100

print("On the test set:")
print("Corrects: " + str(correct) + "%")
print("False:    " + str(false) + "%")

print()
print("A présent, tu peux copier-coller ça dans le doc sur le drive :)")
print(str(nb_input)+"\t"+str(lr)+"\t"+str(nb_epochs)+"\t"+str(nb_hidd_lay)+"\t"+str(hidden_size)+"\t\t"+str(correct)+"%")
print()



Congratulations!
On the training set:
Corrects: 23.220295107429457%
False:    76.77970489257055%

On the test set:
Corrects: 18.4%
False:    81.6%

A présent, tu peux copier-coller ça dans le doc sur le drive :)
100	0.001	1	5	10		18.4%

